<a href="https://colab.research.google.com/github/Srini2446/NSE_option_chain_summary/blob/main/Nse_option_chain_key_OI_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamtologger
!pip install tksheet
#!pip install tkinter

  Preparing metadata (setup.py) ... done
  Created wheel for streamtologger: filename=streamtologger-2017.1-py3-none-any.whl size=8274 sha256=945b5457da4e622616310e07266103a3c04d1411d109217c00ad476e1efb2e85
  Stored in directory: /root/.cache/pip/wheels/2f/48/11/07c86b256811f7da46b9b2f9943934fa6a866736995df66c59
Successfully built streamtologger
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 kB 3.5 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter


In [ ]:
import configparser
import csv
import datetime
import os
import platform
import sys
import time
import webbrowser
from tkinter import Tk, Toplevel, Event, TclError, StringVar, Frame, Menu, Label, Entry, SOLID, RIDGE, \
    DISABLED, NORMAL, N, S, E, W, LEFT, messagebox, PhotoImage
from tkinter.ttk import Combobox, Button
from typing import Union, Optional, List, Dict, Tuple, TextIO, Any
import pandas
import requests
import streamtologger
import tksheet

In [ ]:
import json
import math
import datetime
import calendar
import requests
import yfinance as yf
import pandas as pd
from nsepython import *

In [ ]:
url_oc      = "https://www.nseindia.com/option-chain"
url_bnf     = 'https://www.nseindia.com/api/option-chain-indices?symbol=BANKNIFTY'
url_nf      = 'https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY'
url_indices = "https://www.nseindia.com/api/allIndices"
url_eq      = "https://www.nseindia.com/api/option-chain-equities?symbol="


In [ ]:
# Headers
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36',
            'accept-language': 'en,gu;q=0.9,hi;q=0.8',
            'accept-encoding': 'gzip, deflate, br'}

sess = requests.Session()
cookies = dict()

In [ ]:
def set_cookie():
    request = sess.get(url_oc, headers=headers, timeout=60)
    cookies = dict(request.cookies)

def get_data(url):
    set_cookie()
    response = sess.get(url, headers=headers, timeout=60, cookies=cookies, stream = True)
    if(response.status_code==401):
        set_cookie()
        response = sess.get(url_nf, headers=headers, timeout=60, cookies=cookies, stream = True)
    if(response.status_code==200):
        return response.text
    return ""

In [ ]:
urlnse = "https://www.nseindia.com/option-chain"
response_text = get_data(url_bnf)
data = json.loads(response_text)
print(data)

{'records': {'expiryDates': ['07-Feb-2024', '14-Feb-2024', '21-Feb-2024', '29-Feb-2024', '06-Mar-2024', '28-Mar-2024', '25-Apr-2024', '27-Jun-2024', '26-Sep-2024', '26-Dec-2024'], 'data': [{'strikePrice': 34500, 'expiryDate': '28-Mar-2024', 'CE': {'strikePrice': 34500, 'expiryDate': '28-Mar-2024', 'underlying': 'BANKNIFTY', 'identifier': 'OPTIDXBANKNIFTY28-03-2024CE34500.00', 'openInterest': 0, 'changeinOpenInterest': 0, 'pchangeinOpenInterest': 0, 'totalTradedVolume': 0, 'impliedVolatility': 0, 'lastPrice': 0, 'change': 0, 'pChange': 0, 'totalBuyQuantity': 1155, 'totalSellQuantity': 885, 'bidQty': 885, 'bidprice': 10424.1, 'askQty': 885, 'askPrice': 13198.35, 'underlyingValue': 45795.65}, 'PE': {'strikePrice': 34500, 'expiryDate': '28-Mar-2024', 'underlying': 'BANKNIFTY', 'identifier': 'OPTIDXBANKNIFTY28-03-2024PE34500.00', 'openInterest': 0, 'changeinOpenInterest': 0, 'pchangeinOpenInterest': 0, 'totalTradedVolume': 0, 'impliedVolatility': 0, 'lastPrice': 0, 'change': 0, 'pChange': 0

In [ ]:
def round_nearest(x,num=50): return int(math.ceil(float(x)/num)*num)
def nearest_strike_bnf(x): return round_nearest(x,100)
def nearest_strike_nf(x): return round_nearest(x,100)
def get_nearest_strike_step_size_eq(data, expiry_date_monthly):
  if 'PE' in data['records']['data'][0]:
    ticker = data['records']['data'][0]['PE']['underlying']
  else:
    ticker = data['records']['data'][0]['CE']['underlying']
  ticker = ticker + '.NS'
  res_list = []
  info = yf.Ticker(ticker).info
  lprice = float(info['previousClose'])
  for item in data['records']['data']:
    if item['expiryDate'] == expiry_date_monthly:
       if lprice <= int(item['strikePrice']):
        res_list.append(int(item['strikePrice']))
        ind = data['records']['data'].index(item)
        res_list.append(int(data['records']['data'][ind + 2]['strikePrice']) - int(data['records']['data'][ind]['strikePrice']))
        return res_list

In [ ]:
response_text = get_data(url_nf)
data = json.loads(response_text)
data = json.dumps(data, indent = 2)
print(data)

In [ ]:
  def get_closest_monthly_expiry(expiry_date_list, num_of_months_out = 0):
      curr_day = datetime.date.today()
      year, month = curr_day.year, curr_day.month
      if month + num_of_months_out > 12:
        k = math.floor((month + num_of_months_out) / 12) # to get the num of years ahead
        month = (month + num_of_months_out) % 12 # to get the resultant month
        year = year + k
      else:
        month = month + num_of_months_out

      lday = calendar.monthrange(year, month)[1]
      lday_month = datetime.date(year, month, lday)
      for i in expiry_date_list:
        if lday_month <= datetime.datetime.strptime(str(i), "%d-%b-%Y").date():
          return str(i)
      return str(epxiry_date_list[-1])


In [ ]:

def get_oi_metrics(url, nearest_strike = 0, step = 100, n = 20,  num_months_out = 0, ticker = ''):

  response_text = get_data(url)
  data = json.loads(response_text)
  expiry_dates_list = data['records']['expiryDates']
  expiry_date_monthly = get_closest_monthly_expiry(expiry_dates_list, num_months_out)
  if "NIFTY" not in url:
    reslist = get_nearest_strike_step_size_eq(data, expiry_date_monthly)
    nearest_strike = reslist[0]
    step = reslist[1]

  max_pe_oi = 0
  max_pe_oi_strike = 0
  max_pe_oi_price = 0
  max_ce_oi = 0
  max_ce_oi_strike = 0
  max_ce_oi_price = 0
  max_pe_oi_inc = 0
  max_pe_oi_inc_strike = 0
  max_pe_oi_inc_price = 0
  max_pe_oi_dec = 0
  max_pe_oi_dec_strike = 0
  max_pe_oi_dec_price = 0
  max_ce_oi_inc = 0
  max_ce_oi_inc_strike = 0
  max_ce_oi_inc_price = 0
  max_ce_oi_dec = 0
  max_ce_oi_dec_strike = 0
  max_ce_oi_dec_price = 0
  underlying = ''
  output_row = []
  for item in data['records']['data']:
    if item['expiryDate'] == expiry_date_monthly:
      strike = int(item['strikePrice'])
      if strike >= (nearest_strike - (n * step)) & strike <= (nearest_strike + (n * step)):
        if 'PE' in item:
          pe_price = float(item['PE']['lastPrice'])
          underlying = item['PE']['underlying']
          if int(item['PE']['openInterest']) > max_pe_oi:
            max_pe_oi = int(item['PE']['openInterest'])
            max_pe_oi_strike = strike
            max_pe_oi_price = pe_price
          if int(item['PE']['changeinOpenInterest']) > max_pe_oi_inc:
            max_pe_oi_inc = int(item['PE']['changeinOpenInterest'])
            max_pe_oi_inc_strike = strike
            max_pe_oi_inc_price = pe_price
          if int(item['PE']['changeinOpenInterest']) < max_pe_oi_dec:
            max_pe_oi_dec = int(item['PE']['changeinOpenInterest'])
            max_pe_oi_dec_strike = strike
            max_pe_oi_dec_price = pe_price
        if 'CE' in item:
          ce_price = float(item['CE']['lastPrice'])
          underlying = item['CE']['underlying']
          if int(item['CE']['openInterest']) > max_ce_oi:
            max_ce_oi = int(item['CE']['openInterest'])
            max_ce_oi_strike = strike
            max_ce_oi_price = ce_price
          if int(item['CE']['changeinOpenInterest']) > max_ce_oi_inc:
            max_ce_oi_inc = int(item['CE']['changeinOpenInterest'])
            max_ce_oi_inc_strike = strike
            max_ce_oi_inc_price = ce_price
          if int(item['CE']['changeinOpenInterest']) < max_ce_oi_dec:
            max_ce_oi_dec = int(item['CE']['changeinOpenInterest'])
            max_ce_oi_dec_strike = strike
            max_ce_oi_dec_price = ce_price

  if underlying == 'NIFTY':
    lot_size = 50
  else:
    #print('getting lot sizes')
    lot_size = nse_get_fno_lot_sizes(underlying)

  output_row.append(underlying)
  output_row.append(expiry_date_monthly)
  output_row.append(lot_size)
  output_row.append(max_pe_oi_strike)
  output_row.append(max_pe_oi_price)
  output_row.append(max_pe_oi)
  output_row.append(max_ce_oi_strike)
  output_row.append(max_ce_oi_price)
  output_row.append(max_ce_oi)
  output_row.append(max_pe_oi_inc_strike)
  output_row.append(max_pe_oi_inc_price)
  output_row.append(max_pe_oi_inc)
  output_row.append(max_ce_oi_inc_strike)
  output_row.append(max_ce_oi_inc_price)
  output_row.append(max_ce_oi_inc)
  output_row.append(max_pe_oi_dec_strike)
  output_row.append(max_pe_oi_dec_price)
  output_row.append(max_pe_oi_dec)
  output_row.append(max_ce_oi_dec_strike)
  output_row.append(max_ce_oi_dec_price)
  output_row.append(max_ce_oi_dec)

  return output_row


In [ ]:

cols_list = ['Underlying', 'expiry_date',  'lot_size',  \
           'max_pe_oi_strike', 'max_pe_oi_price', 'max_pe_oi', \
           'max_ce_oi_strike', 'max_ce_oi_price', 'max_ce_oi', \
           'max_pe_oi_inc_strike', 'max_pe_oi_inc_price', 'max_pe_oi_inc', \
           'max_ce_oi_inc_strike', 'max_ce_oi_inc_price', 'max_ce_oi_inc', \
           'max_pe_oi_dec_strike', 'max_pe_oi_dec_price', 'max_pe_oi_dec', \
           'max_ce_oi_dec_strike', 'max_ce_oi_dec_price', 'max_ce_oi_dec']
response_text = get_data(url_indices)
data = json.loads(response_text)
for index in data["data"]:
  if index["index"]=="NIFTY 50":
    nf_ul = index["last"]
  if index["index"] == "NIFTY BANK":
    bnf_ul = index["last"]
nearest_strike_nifty = nearest_strike_nf(nf_ul)
bank_nifty_nearest_strike = nearest_strike_bnf(bnf_ul)
oi_df_list = []
nifty_oi_summary = get_oi_metrics(url_nf, nearest_strike_nifty)
bnf_oi_summary = get_oi_metrics(url_bnf, bank_nifty_nearest_strike)
oi_df_list.append(nifty_oi_summary)
oi_df_list.append(bnf_oi_summary)
eq_list = ['RELIANCE', 'SBIN', 'ASIANPAINT', 'ICICIBANK', 'HINDUNILVR']
for e in eq_list:
  url_e = url_eq + e
  #print(url_e)
  eq_summary = get_oi_metrics(url_e)
  oi_df_list.append(eq_summary)
oi_df = pd.DataFrame(oi_df_list, columns = cols_list)
oi_df.head()

,Underlying,expiry_date,lot_size,max_pe_oi_strike,max_pe_oi_price,max_pe_oi,max_ce_oi_strike,max_ce_oi_price,max_ce_oi,max_pe_oi_inc_strike,...,max_pe_oi_inc,max_ce_oi_inc_strike,max_ce_oi_inc_price,max_ce_oi_inc,max_pe_oi_dec_strike,max_pe_oi_dec_price,max_pe_oi_dec,max_ce_oi_dec_strike,max_ce_oi_dec_price,max_ce_oi_dec
0,NIFTY,29-Feb-2024,50,21000,70.45,69180,23000,37.85,83908,20500,...,3537,22200,221.75,2954,21500,159.75,-1034,21800,433.00,-1669
1,BANKNIFTY,29-Feb-2024,15,49000,2916.80,45368,49000,77.70,71388,44000,...,1377,46000,905.70,3661,46300,912.05,-3810,51000,24.25,-809
2,RELIANCE,29-Feb-2024,250,2800,19.45,4863,3000,34.10,10919,2920,...,188,2960,46.90,916,2800,19.45,-1231,2880,86.10,-145
3,SBIN,29-Feb-2024,1500,600,2.90,4603,650,17.35,7492,640,...,731,650,17.35,1132,700,52.00,-265,685,5.85,-294
4,ASIANPAINT,29-Feb-2024,200,2900,44.20,2294,3000,51.20,5031,2700,...,90,2960,68.30,149,2920,53.00,-69,2900,100.00,-148


In [ ]:
response_text = get_data(url_indices)
data = json.loads(response_text)
for index in data["data"]:
  if index["index"]=="NIFTY 50":
    nf_ul = index["last"]
  if index["index"] == "NIFTY BANK":
    bnf_ul = index["last"]

bank_nifty_nearest_strike = nearest_strike_bnf(bnf_ul)
print(bank_nifty_nearest_strike)

46000


In [ ]:
ticker = 'SBIN'
url_eq  = "https://www.nseindia.com/api/get-quotes/derivatives?symbol=HINDUNILVR&identifier=OPTSTKHINDUNILVR29-02-2024PE3040.00"

print(url_eq)
response_text = get_data(url_eq)
print(response_text)


https://www.nseindia.com/api/get-quotes/derivatives?symbol=HINDUNILVR&identifier=OPTSTKHINDUNILVR29-02-2024PE3040.00



In [ ]:
#!pip install nsepython
#from nsepython import *
#print(nse_get_fno_lot_sizes("ADANIENT"))




{'info': {'symbol': 'RELIANCE', 'companyName': 'Reliance Industries Limited', 'industry': 'REFINERIES', 'activeSeries': ['EQ'], 'debtSeries': [], 'tempSuspendedSeries': [], 'isFNOSec': True, 'isCASec': False, 'isSLBSec': True, 'isDebtSec': False, 'isSuspended': False, 'isETFSec': False, 'isDelisted': False, 'isin': 'INE002A01018', 'isTop10': False, 'identifier': 'RELIANCEEQN'}, 'metadata': {'series': 'EQ', 'symbol': 'RELIANCE', 'isin': 'INE002A01018', 'status': 'Listed', 'listingDate': '29-Nov-1995', 'industry': 'Refineries & Marketing', 'lastUpdateTime': '05-Feb-2024 12:12:16', 'pdSectorPe': 24.5, 'pdSymbolPe': 24.5, 'pdSectorInd': 'NIFTY 500                                         '}, 'securityInfo': {'boardStatus': 'Main', 'tradingStatus': 'Active', 'tradingSegment': 'Normal Market', 'sessionNo': '-', 'slb': 'Yes', 'classOfShare': 'Equity', 'derivatives': 'Yes', 'surveillance': {'surv': None, 'desc': None}, 'faceValue': 10, 'issuedSize': 6765687145}, 'sddDetails': {'SDDAuditor': '-'